In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
img_width, img_height = 224, 224

2022-07-31 12:29:38.827734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 12:29:38.827769: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_data_dir = 'chest_xray/train'
validation_data_dir = 'chest_xray/val'
nb_train_samples =5216
nb_validation_samples =16
epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [4]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

2022-07-31 12:29:41.617423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-31 12:29:41.617476: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-31 12:29:41.617508: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (janani-Vostro-15-3568): /proc/driver/nvidia/version does not exist
2022-07-31 12:29:41.617911: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width, img_height),batch_size=batch_size,class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,target_size=(img_width, img_height),batch_size=batch_size,class_mode='binary')

model.fit_generator(train_generator,steps_per_epoch=nb_train_samples // batch_size,epochs=epochs,validation_data=validation_generator,validation_steps=nb_validation_samples // batch_size)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


/tmp/ipykernel_23083/3616972421.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,steps_per_epoch=nb_train_samples // batch_size,epochs=epochs,validation_data=validation_generator,validation_steps=nb_validation_samples // batch_size)


Epoch 1/10


2022-07-31 12:29:45.540517: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 101844992 exceeds 10% of free system memory.
2022-07-31 12:29:46.620679: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50466816 exceeds 10% of free system memory.
2022-07-31 12:29:46.623526: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 101844992 exceeds 10% of free system memory.


  1/326 [..............................] - ETA: 22:15 - loss: 0.7227 - accuracy: 0.3750

2022-07-31 12:29:47.509261: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 101844992 exceeds 10% of free system memory.
2022-07-31 12:29:48.184899: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50466816 exceeds 10% of free system memory.


326/326 [==============================] - 300s 910ms/step - loss: 0.3808 - accuracy: 0.8422 - val_loss: 0.9658 - val_accuracy: 0.6250
Epoch 2/10
326/326 [==============================] - 247s 758ms/step - loss: 0.2470 - accuracy: 0.9036 - val_loss: 0.6999 - val_accuracy: 0.6875
Epoch 3/10
326/326 [==============================] - 205s 627ms/step - loss: 0.2196 - accuracy: 0.9206 - val_loss: 0.6306 - val_accuracy: 0.6250
Epoch 4/10
326/326 [==============================] - 186s 568ms/step - loss: 0.2003 - accuracy: 0.9294 - val_loss: 1.7226 - val_accuracy: 0.6250
Epoch 5/10
326/326 [==============================] - 186s 568ms/step - loss: 0.1960 - accuracy: 0.9340 - val_loss: 1.2912 - val_accuracy: 0.6250
Epoch 6/10
326/326 [==============================] - 184s 564ms/step - loss: 0.1891 - accuracy: 0.9337 - val_loss: 0.4676 - val_accuracy: 0.6875
Epoch 7/10
326/326 [==============================] - 2051s 6s/step - loss: 0.1831 - accuracy: 0.9387 - val_loss: 0.5980 - val_accuracy

In [7]:
model.save('model_saved.h5')

In [8]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
from keras.models import load_model
model1 = load_model('model_saved.h5')
image = load_img('chest_xray/test/NORMAL/IM-0001-0001.jpeg', target_size=(224, 224))
img = np.array(image)
print(img.shape)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = (model1.predict(img) > 0.5).astype('int32')
result="Pneumonia" if label else "Normal"
print(result)

(224, 224, 3)
Normal
